In [2]:
# !pip install yahooquery[premium]
# !pip uninstall selenium
# !pip install selenium==3.5.0
import yahooquery
import requests
import json
from yahooquery import Ticker
import time
import json
import requests
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import datetime

import selenium

from selenium import webdriver

In [5]:
def tradingView(
    mktCapMin = 5000000000,
    div_yield_recent = 2,
    StochK = 25,
    StochD = 25,
    macd_macd = 0,
    macd_signal = 0,
):
    filter = [
        {
            "left": "volume",
            "operation": "nempty"
        },
        {
            "left": "type",
            "operation": "in_range",
            "right": [
                "stock",
                "dr",
                "fund"
            ]
        },
        {
            "left": "subtype",
            "operation": "in_range",
            "right": [
                "common",
                "foreign-issuer",
                "",
                "etf",
                "etf,odd",
                "etf,otc",
                "etf,cfd"
            ]
        },
        {
            "left": "exchange",
            "operation": "in_range",
            "right": [
                "AMEX",
                "NASDAQ",
                "NYSE"
            ]
        },
        {
            "left": "market_cap_basic",
            "operation": "egreater",
            "right": mktCapMin
        },
        {
            "left": "is_primary",
            "operation": "equal",
            "right": True
        },
        {
            "left": "Stoch.K",
            "operation": "less",
            "right": StochK
        },
        {
            "left": "Stoch.D",
            "operation": "less",
            "right": StochD
        },
        {
            "left": "MACD.macd",
            "operation": "less",
            "right": macd_macd
        },
        {
            "left": "MACD.signal",
            "operation": "less",
            "right": macd_signal
        },
        {
            "left": "dividend_yield_recent",
            "operation": "egreater",
            "right": div_yield_recent
        },
        {
            "left": "Stoch.RSI.K",
            "operation": "less",
            "right": 25
        },
        {
            "left": "Stoch.RSI.D",
            "operation": "less",
            "right": 25
        }
    ]
    options = {
        "lang": "en"
    }
    markets = {
        "america"
    }
    symbols = {
        "query": {
            "types": []
        },
        "tickers": []
    }
    columns = [
        #"logoid",
        "name",
        "description",
        "close",
        "change",
        "change_abs",
        "Recommend.All",
        "market_cap_basic",
        "price_earnings_ttm",
        "earnings_per_share_basic_ttm",
        "sector",
        "earnings_release_date",
        "earnings_release_next_date",
        "dividend_yield_recent"
    ]
    sort = {
        "sortBy": "volume",
        "sortOrder": "desc"
    }
    range = [
        0,
        150
    ]

    post_message = {}
    post_message['filter'] = filter
    post_message['options'] = options
    post_message['symbols'] = symbols
    post_message['columns'] = columns
    post_message['sort'] = sort
    post_message['range'] = range

    payload = json.dumps(post_message)


    url = "https://scanner.tradingview.com/america/scan"
    #     payload = self.payload
    headers = {
        'authority': 'scanner.tradingview.com',
        'accept': 'text/plain, */*; q=0.01',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.9',
        'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'dnt': '1',
        'origin': 'https//www.tradingview.com',
        'referer': 'https//www.tradingview.com/',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="96", "Google Chrome";v="96"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    response_json = response.json()['data']
    if not response_json:
        return []
    df0 = pd.DataFrame.from_dict(response_json)
    df0.drop(columns=['s'], inplace=True)
    df = pd.DataFrame(df0["d"].to_list(), columns=columns)
#     display(df)

    df['change'] = df['change'].round(2)
    df['dividend_yield_recent'] = df['dividend_yield_recent'].round(2)
    df['price_earnings_ttm'] = df['price_earnings_ttm'].round(2)
    df['earnings_per_share_basic_ttm'] = df['earnings_per_share_basic_ttm'].round(2)
#     display(df)
    myListOfTickers = df['name'].to_list()
#     executionTime = (time.time() - startTime)
#     tradingViewTime = tradingViewTime + executionTime

    # return render(request, 'options/options.html', results)
    return myListOfTickers



In [6]:
tvlist = tradingView()
display(tvlist)

yf_username = 'qcappartners@yahoo.com'
yf_password = 'Mcci2022!!'
# 'DilutedEPS', 'BasicEPS'
fin_types = ['DilutedEPS', 'BasicEPS']
types = ['TotalDebt', 'TotalAssets', 'EBIT', 'EBITDA', 'PeRatio']
financial_data = Ticker(tvlist, username=yf_username, password=yf_password).p_get_financial_data(types=types, trailing=False)
display(financial_data)

['F',
 'WFC',
 'HBAN',
 'HAL',
 'MRK',
 'M',
 'VTRS',
 'BKR',
 'WBA',
 'PR',
 'PSX',
 'SYF',
 'IVZ',
 'APO',
 'EPD',
 'JCI',
 'UNP',
 'BEN',
 'CAT',
 'OKE',
 'CF',
 'DFS',
 'TRGP',
 'CRBG',
 'SWK',
 'NSC',
 'TAP',
 'AAP',
 'CE',
 'LHX',
 'CHRW',
 'CMI',
 'APD',
 'BC',
 'PII',
 'TPG',
 'BR',
 'VAC']

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 109
Current browser version is 111.0.5563.65 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	(No symbol) [0x00E96643]
	(No symbol) [0x00E2BE21]
	(No symbol) [0x00D2DA9D]
	(No symbol) [0x00D52911]
	(No symbol) [0x00D4D630]
	(No symbol) [0x00D4A899]
	(No symbol) [0x00D86917]
	(No symbol) [0x00D8655C]
	(No symbol) [0x00D7FB76]
	(No symbol) [0x00D549C1]
	(No symbol) [0x00D55E5D]
	GetHandleVerifier [0x0110A142+2497106]
	GetHandleVerifier [0x011385D3+2686691]
	GetHandleVerifier [0x0113BB9C+2700460]
	GetHandleVerifier [0x00F43B10+635936]
	(No symbol) [0x00E34A1F]
	(No symbol) [0x00E3A418]
	(No symbol) [0x00E3A505]
	(No symbol) [0x00E4508B]
	BaseThreadInitThunk [0x75A37D69+25]
	RtlInitializeExceptionChain [0x775CB74B+107]
	RtlClearBits [0x775CB6CF+191]


In [4]:
# display(financial_data)
print(len(tvlist))
data = financial_data
data = data.reset_index()
print(len(data['symbol'].unique()))

14
14


In [5]:
def get_price(ticker):

    tick = Ticker(ticker)
    raw_dict = tick.price
    df = pd.DataFrame.from_dict(raw_dict)
    return raw_dict
df = get_price('aapl')
display(df)
print(df['aapl']['regularMarketPrice'])
print(df['aapl']['marketCap'])

{'aapl': {'maxAge': 1,
  'preMarketChangePercent': -0.005204626,
  'preMarketChange': -0.80000305,
  'preMarketTime': '2023-02-17 07:19:57',
  'preMarketPrice': 152.91,
  'preMarketSource': 'FREE_REALTIME',
  'postMarketChangePercent': -0.0052371863,
  'postMarketChange': -0.80500793,
  'postMarketTime': '2023-02-16 17:59:59',
  'postMarketPrice': 152.905,
  'postMarketSource': 'DELAYED',
  'regularMarketChangePercent': -0.010429377,
  'regularMarketChange': -1.6199951,
  'regularMarketTime': '2023-02-16 14:00:05',
  'priceHint': 2,
  'regularMarketPrice': 153.71,
  'regularMarketDayHigh': 156.3299,
  'regularMarketDayLow': 153.3475,
  'regularMarketVolume': 66914188,
  'regularMarketPreviousClose': 155.33,
  'regularMarketSource': 'FREE_REALTIME',
  'regularMarketOpen': 153.51,
  'exchange': 'NMS',
  'exchangeName': 'NasdaqGS',
  'exchangeDataDelayedBy': 0,
  'marketState': 'PRE',
  'quoteType': 'EQUITY',
  'symbol': 'AAPL',
  'underlyingSymbol': None,
  'shortName': 'Apple Inc.',
  '

153.71
2431984336896
